In [29]:
import duckdb

%load_ext sql
conn = duckdb.connect("../rewards_dbt/rewards_db.duckdb")
%sql conn --alias duckdb
%config SqlMagic.displaylimit = 15

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [30]:
%%sql
select * exclude(rewards_receipt_item_list)
from
    core.receipts
order by
    1

Running query in 'duckdb'

id,bonus_points_earned,bonus_points_earned_reason,create_date,scanned_date,finished_date,modify_date,points_awarded_date,points_earned,purchase_date,purchase_item_count,rewards_receipt_status,total_spent,user_id
5f9c74f70a7214ad07000037,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-10-30 20:17:59,2020-10-30 20:17:59,None,2021-01-03 15:24:54,2020-10-30 20:18:00,750.00,2020-10-29 20:17:59,11,REJECTED,1.00,5f9c74f7c88c1415cbddb839
5f9c74f90a7214ad07000038,300,"Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-10-30 20:18:01,2020-10-30 20:18:01,2021-01-03 15:39:55,2021-01-03 15:39:55,2021-01-03 15:39:55,389.20,2020-10-29 20:18:01,6,FINISHED,14.00,5f9c74e3f1937815bd2c1d73
5fa5ad370a720f05ef000089,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-11-06 20:08:23,2020-11-06 20:08:23,2021-01-05 20:53:40,2021-01-05 20:53:40,2021-01-05 20:53:40,9449.80,2020-11-05 20:08:23,11,FINISHED,291.00,5fa5ad376a26f611e71ab5ef
5fa5b0ca0a720f05ef0000bf,100,"Receipt number 6 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-11-05 20:23:38,2020-11-05 20:23:38,2021-01-08 21:44:00,2021-01-08 21:44:00,2021-01-08 21:44:00,189.20,2020-11-04 20:23:38,6,FINISHED,14.00,5fa5b0b720dc5111dd86dcc1
5fa8d5730a7214adc30001c3,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-11-09 05:36:51,2020-11-09 05:36:51,None,2021-01-27 01:13:20,2020-11-09 05:36:52,750.00,2020-11-08 05:36:51,11,REJECTED,1.00,5fa8d573770b516dda4f4d21
5fa8d5790a7214adc30001c4,5,All-receipts receipt bonus,2020-11-09 05:36:57,2020-11-09 05:36:57,2021-01-27 01:28:21,2021-01-27 01:28:21,2021-01-27 01:28:21,94.60,2020-11-08 05:36:57,6,FINISHED,14.00,5fa8d542b055fd1dfdf09517
5fb1bd510a7214ad890000e3,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-11-15 23:44:17,2020-11-15 23:44:17,None,2021-02-02 00:28:24,2020-11-15 23:44:17,750.00,2020-11-14 23:44:17,11,REJECTED,1.00,5fb1bd519c09d811e5211cef
5fb2da580a7214ad89000114,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2020-11-16 20:00:24,2020-11-16 20:00:24,None,2021-02-02 00:28:24,2020-11-16 20:00:24,750.00,2020-11-15 20:00:24,11,REJECTED,1.00,5fb2da582357ec11d4ecc403
5ff1e1960a720f0523000567,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:06,2021-01-03 15:24:06,2021-01-03 15:24:10,2021-01-03 15:24:10,2021-01-03 15:24:10,750.00,2021-01-02 15:24:06,1,FINISHED,1.00,5ff1e196cfcf6c399c274a38
5ff1e1a10a720f0523000568,750,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:17,2021-01-03 15:24:17,2021-01-03 15:39:20,2021-01-03 15:39:20,2021-01-03 15:24:19,750.00,2021-01-02 00:00:00,5,FINISHED,50.00,5ff1e194b6a9d73a3a9f1052


In [14]:
%%sql
select
    rewards_receipt_status,
    count(0) as count
from
    core.receipts
group by
    1

Running query in 'duckdb'

rewards_receipt_status,count
FINISHED,518
SUBMITTED,434
PENDING,50
REJECTED,71
FLAGGED,46


In [15]:
%%sql
select
    case rewards_receipt_status
        when 'FINISHED' then 'ACCEPTED'
        else rewards_receipt_status
    end as status,
    round(avg(total_spent)) as avg_spend
from
    core.receipts
where
    status in ('ACCEPTED', 'REJECTED')
group by
    1


Running query in 'duckdb'

status,avg_spend
REJECTED,23.0
ACCEPTED,81.0


With the results above, we see that the average spend from receipts with `rewards_receipt_status` value of Accepted(Finished) and Rejected. This value comes with the assumption that a value of `FINISHED` is a synonum for `ACCEPTED`. This assumption needs to be verified with the product owner/business to verify. If this assumption is wrong, more logic will be necessary to produce an accurate value.

I also assume that the status values of flagged, pending, and submitted do not fall within the definition of accepted or rejected. This is also another clarification to follow up on.

---
#### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [16]:
%%sql
select
    case rewards_receipt_status
        when 'FINISHED' then 'ACCEPTED'
        else rewards_receipt_status
    end as status,
    sum(purchase_item_count) as total_items_purchased
from
    core.receipts
where
    status in ('ACCEPTED', 'REJECTED')
group by
    1

Running query in 'duckdb'

status,total_items_purchased
REJECTED,173
ACCEPTED,8184


With the same assumptions in place as listed previously, the total number of items purchased for `ACCPETED` items is much more than `REJECTED`.

---
Which brand has the most spend among users who were created within the past 6 months?

To get the brand, we need to join with the brands table using either:
- barcode
- brand_code

Since we're looking for total spend and not rewards receipt status, all statuses will be considered.

In [18]:
%%sql
select
    *
from
    core.receipts_item_list

Running query in 'duckdb'

receipt_id,barcode,brand_code,competitor_rewards_group,is_competitive_product,description,discounted_item_price,final_price,item_number,item_price,needs_fetch_review,needs_fetch_review_reason,original_final_price,original_metabrite_barcode,metabrite_campaign_id,original_metabrite_description,original_meta_brite_item_price,original_metabrite_quantity_purchased,original_receipt_item_text,partner_item_id,points_earned,points_payer_id,points_not_awarded_reason,prevent_target_gap_points,price_after_coupon,quantity_purchased,rewards_group,rewards_product_partner_id,target_price,user_flagged_barcode,user_flagged_description,is_user_flagged_new_item,user_flagged_price,user_flagged_quantity,is_deleted
5ff1e1eb0a720f0523000575,4011,None,None,None,ITEM NOT FOUND,None,26.00,None,26.00,False,None,None,None,None,None,None,None,None,1,None,None,None,True,None,5,None,None,None,4011,None,True,26.00,5,None
5ff1e1bb0a720f052300056b,4011,None,None,None,ITEM NOT FOUND,None,1,None,1,None,None,None,None,None,None,None,None,None,1,None,None,None,None,None,1,None,None,None,None,None,None,None,None,None
5ff1e1bb0a720f052300056b,028400642255,None,None,None,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ,None,10.00,None,10.00,True,USER_FLAGGED,None,None,None,None,None,None,None,2,None,5332f5fbe4b03c9a25efd0ba,Action not allowed for user and CPG,True,None,1,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,None,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ,True,10.00,1,None
5ff1e1f10a720f052300057a,None,None,None,None,None,None,None,None,None,False,None,None,None,None,None,None,None,None,1,None,None,None,True,None,None,None,None,None,4011,None,True,26.00,3,None
5ff1e1ee0a7214ada100056f,4011,None,None,None,ITEM NOT FOUND,None,28.00,None,28.00,False,None,None,None,None,None,None,None,None,1,None,None,None,True,None,4,None,None,None,4011,None,True,28.00,4,None
5ff1e1d20a7214ada1000561,4011,None,None,None,ITEM NOT FOUND,None,1,None,1,None,None,None,None,None,None,None,None,None,1,None,None,None,None,None,1,None,None,None,None,None,None,None,None,None
5ff1e1d20a7214ada1000561,1234,None,None,None,None,None,2.56,None,2.56,True,USER_FLAGGED,None,None,None,None,None,None,None,2,None,None,None,True,None,3,None,None,None,1234,,True,2.56,3,None
5ff1e1e40a7214ada1000566,4011,None,None,None,ITEM NOT FOUND,None,3.25,None,3.25,False,None,None,028400642255,None,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ,None,None,None,1,None,5332f5fbe4b03c9a25efd0ba,Action not allowed for user and CPG,True,None,1,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,None,4011,None,None,None,None,None
5ff1e1cd0a720f052300056f,None,MISSION,TACO BELL TACO SHELLS,None,MSSN TORTLLA,2.23,2.23,None,2.23,None,None,None,None,None,None,None,None,MSSN TORTLLA,1009,None,None,None,None,None,1,None,None,None,None,None,None,None,None,None
5ff1e1a40a720f0523000569,046000832517,BRAND,None,None,"Old El Paso Mild Chopped Green Chiles, 4.5 Oz",None,10.00,None,10.00,None,None,None,None,None,None,None,None,None,0,None,5332f5f3e4b03c9a25efd0ae,Action not allowed for user and CPG,None,None,1,OLD EL PASO BEANS & PEPPERS,5332f5f3e4b03c9a25efd0ae,None,None,None,None,None,None,None


Filters
- Remove rows where we don't know the `barcode` or the `brand_code`

In [31]:
%%sql
select
    ril.receipt_id,
    ril.barcode,
    ril.brand_code,
    coalesce(ril.user_flagged_price, ril.item_price) as price,
    ril.item_price,
    ril.user_flagged_price
from
    core.receipts_item_list as ril
where
    -- filter out items if we don't know the brand or barcode
    barcode is not null
    or brand_code is not null
order by
    receipt_id

Running query in 'duckdb'

receipt_id,barcode,brand_code,price,item_price,user_flagged_price
5f9c74f70a7214ad07000037,075925306254,None,1,1,None
5f9c74f90a7214ad07000038,034100573065,None,1.00,1.00,1.00
5f9c74f90a7214ad07000038,034100573065,None,1.00,1.00,1.00
5f9c74f90a7214ad07000038,034100573065,None,1.00,1.00,1.00
5f9c74f90a7214ad07000038,075925306254,None,10.00,10.00,10.00
5f9c74f90a7214ad07000038,075925306254,None,1,1,None
5fa5ad370a720f05ef000089,075925306254,None,1,1,None
5fa5ad370a720f05ef000089,034100573065,None,29.00,29.00,29.00
5fa5ad370a720f05ef000089,034100573065,None,29.00,29.00,29.00
5fa5ad370a720f05ef000089,034100573065,None,29.00,29.00,29.00
